# Model training and evaluation

In [6]:
import os
import sys
import glob
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50
import matplotlib.pyplot as plt
import seaborn as sns

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # Hide warning logs from tensorflow
tf.get_logger().setLevel("ERROR")

In [ ]:
# Constants
IMG_SIZE = 224
BATCH_SIZE = 300
AUTOTUNE = tf.data.AUTOTUNE
EPOCHS = 5
INPUT_SHAPE=(224, 224, 3)

tf.random.set_seed(5)
dataset_dir = "../datasets"

# Change dataset_dir when run in google colab 
if 'google.colab' in sys.modules:
    from google.colab import drive

    drive.mount('/content/drive')
    dataset_dir = "/content/drive/Othercomputers/Big Mac/datasets"

physical_gpus = tf.config.list_physical_devices('GPU')
print("Using available GPUs: ", physical_gpus)

tf.keras.mixed_precision.set_global_policy('float32')

Using available GPUs:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
# Load clean datasets
def prepare_input_data(input):
    image = tf.cast(input['image'], tf.float32)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = preprocess_input(image)
    label = input['label']
    return image, label

(train, validation, test), info = tfds.load(
    'imagenette',
    split=['train', 'validation[:50%]', 'validation[50%:]'],
    shuffle_files=True,
    with_info=True,
    data_dir=dataset_dir
)

# (train, validation, test), info = tfds.load(
#     'imagenet2012',
#     shuffle_files=False,
#     with_info=True,
#     data_dir=dataset_dir
# )

print(f'Train image count: {info.splits["train"].num_examples}')
print(f'Validation image count: {info.splits["validation[:50%]"].num_examples}')
print(f'Test image count: {info.splits["validation[50%:]"].num_examples}')

train_dataset = train.map(prepare_input_data, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
validation_dataset = validation.map(prepare_input_data, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
test_dataset = test.map(prepare_input_data, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

Train image count: 9469
Validation image count: 1962
Test image count: 1963


In [ ]:
# Load adversarial datasets

def _parse_function(proto):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    parsed_features = tf.io.parse_single_example(proto, feature_description)
    image_f16 = tf.io.parse_tensor(parsed_features['image'], out_type=tf.float16)
    label = parsed_features['label']
    image_f32 = tf.cast(image_f16, tf.float32)
    image_f32.set_shape([IMG_SIZE, IMG_SIZE, 3])
    return image_f32, label


# Get all test and train tfrec files
test_file_paths = glob.glob(f'{dataset_dir}/adversaries/imagenette/test-*.tfrec')
train_file_paths = glob.glob(f'{dataset_dir}/adversaries/imagenette/train-*.tfrec')
print(f"Loaded {len(train_file_paths)} TFrecord train files")
print(f"Loaded {len(test_file_paths)} TFrecord test files")

# Create a TFRecordDataset
raw_test_dataset = tf.data.TFRecordDataset(test_file_paths, compression_type='GZIP')
adv_test_dataset = raw_test_dataset.map(_parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

raw_adv_train_dataset = tf.data.TFRecordDataset(train_file_paths, compression_type='GZIP')
adv_train_dataset = raw_adv_train_dataset.map(_parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

Loaded 8 TFrecord test files
Loaded 19 TFrecord train files


In [ ]:
print("Training robust ResNet-50 model...\n")

robust_model = ResNet50(
    include_top=true,
    weights='imagenet',
    input_shape=INPUT_SHAPE,
    pooling=None,
    classes=10,
    classifier_activation='softmax'
)

robust_model.trainable = False # This prevents the powerful, pre-trained weights from being destroyed by large, random gradients during the first phase of training

for layer in robust_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)      # reduce dimensions
x = Dropout(0.5)(x)                  # help prevent overfitting
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
robust_model.compile(
    optimizer=Adam(learning_rate=0.001), # Using a higher learning rate (~1e-3) to only train top layers.
    loss='sparse_categorical_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='top_5_accuracy'),
    ]
)

In [ ]:
robust_model.fit(adv_train_dataset, verbose=1, batch_size=BATCH_SIZE, epochs=EPOCHS)

In [ ]:
robust_model.save("robust_resnet50.keras")